In [1]:
!pip install selenium

In [1]:
links=[
       'https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile',#KADAMBA
       'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile',#SOUTHBENGAL
       'https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile',#HRTC
       'https://www.redbus.in/online-booking/astc/?utm_source=rtchometile', #ASSAM
       'https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile',#WBTC
       'https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu',#CHANDIGARH
       'https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile',#PUNJAB
       'https://www.redbus.in/online-booking/north-bengal-state-transport-corporation',#NORTHBENGAL
       'https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile',#BIHAR
       'https://www.redbus.in/online-booking/kaac-transport']#KAAC



from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

######################################################NO.0 KADABA##########################################################################3
driver = webdriver.Chrome()
driver.get(links[0])
time.sleep(5)

def get_route_links(driver):
    route_links = []
    try:
        body = driver.find_elements(By.XPATH, "//a[@class='route']")
        route_links = [link.get_attribute('href') for link in body]
    except Exception as e:
        print(f"Error in getting route links: {e}")
    return route_links

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})

def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,5):  # Adjust the range based on the number of pages you want to navigate
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 4:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)


def click_view_buses(driver):
    b=driver.find_elements(By.CSS_SELECTOR,"div[class='button']")
    for i in range(len(b)-1,-1,-1):
        try:
            b[i].click()            
        except Exception as e:
              print()

def scroll_down_page(driver):
    try:
        body = driver.find_element(By.TAG_NAME, "body")
        for _ in range(14):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
    except Exception as e:
        print(f"Error scrolling down the page: {e}")


def extract_bus_data(driver, urls):
    bus_data = []
    for url in urls:
        try:
            driver.get(url)
            time.sleep(5)  # Wait for the page to load
            click_view_buses(driver)
            scroll_down_page(driver)

        
            bus_names = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
            bus_types = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
            departures_time = driver.find_elements(By.XPATH, "//div[contains(@class, 'dp-time f-19 d-color f-bold')]")
            reachings_time = driver.find_elements(By.XPATH, "//div[contains(@class, 'bp-time f-19 d-color disp-Inline')]")
            durations = driver.find_elements(By.XPATH, "//div[contains(@class, 'dur l-color lh-24')]")
            seats_available = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")
            prices = driver.find_elements(By.XPATH, "//div[contains(@class, 'fare d-block')]")
            star_ratings = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating-sec lh-24')]")

            for i in range(len(bus_names)):
                bus_data.append({
                    "route_link": url,
                    "bus_name": bus_names[i].text if i < len(bus_names) else '',
                    "bus_type": bus_types[i].text if i < len(bus_types) else '',
                    "departure_time": departures_time[i].text if i < len(departures_time) else '',
                    "reaching_time": reachings_time[i].text if i < len(reachings_time) else'',
                    "duration": durations[i].text if i < len(durations) else '',
                    "seats_available": seats_available[i].text if i < len(seats_available) else '',
                    "price": prices[i].text if i < len(prices) else '',
                    "star_rating" : star_ratings[i].text if i < len(star_ratings) else ''
                })

        except Exception as e:
             print(f"Error extracting data for URL {url}: {e}")
    return bus_data


bus_data = extract_bus_data(driver, all_links)
for bus in bus_data:
    print(bus)

driver.quit()

bus_dataframe1 = pd.DataFrame(bus_data)
bus_dataframe1.to_csv('bus_dataframe1.csv', index=False)


#################################################################NO.1 SOUTHBENGAL#####################################################################TSRTC


driver = webdriver.Chrome()
driver.get(links[1])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})

def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,6): 
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 5:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

driver.quit()

bus_dataframe2 = pd.DataFrame(bus_data)
bus_dataframe2.to_csv('bus_dataframe2.csv', index=False)


#############################################################NO.2 HRTC#################################################################################
driver = webdriver.Chrome()
driver.get(links[2])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})

def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,6): 
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 5:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)

bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

bus_dataframe3 = pd.DataFrame(bus_data)
bus_dataframe3.to_csv('bus_dataframe3.csv', index=False)

#############################################################NO.3 ASSAM##########################################################################

driver = webdriver.Chrome()
driver.get(links[3])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})

def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,6): 
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 5:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

driver.quit()

bus_dataframe4 = pd.DataFrame(bus_data)
bus_dataframe4.to_csv('bus_dataframe4.csv', index=False)


##########################################################NO.4 WBTC############################################################################

driver = webdriver.Chrome()
driver.get(links[4])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})

def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,6): 
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 5:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)

bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

driver.quit()

bus_dataframe5 = pd.DataFrame(bus_data)
bus_dataframe5.to_csv('bus_dataframe5.csv', index=False)

#############################################################NO.5  CHANDIGARH#############################################################

driver = webdriver.Chrome()
driver.get(links[5])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})


def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,6): 
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 5:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

driver.quit()
bus_dataframe6 = pd.DataFrame(bus_data)
bus_dataframe6.to_csv('bus_dataframe6.csv', index=False)

#############################################################NO.6 PUNJAB#############################################################################

driver = webdriver.Chrome()
driver.get(links[6])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})


def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,4): 
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 3:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

bus_dataframe7 = pd.DataFrame(bus_data)
bus_dataframe7.to_csv('bus_dataframe7.csv', index=False)


#######################################################NO.7 NORTHBENGAL  ###########################################################################


driver = webdriver.Chrome()
driver.get(links[7])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})


def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,6): 
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 5:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)

driver.quit()
bus_dataframe8 = pd.DataFrame(bus_data)
bus_dataframe8.to_csv('bus_dataframe8.csv', index=False)


#################################################################NO.8 BIHAR##########################################################3

driver = webdriver.Chrome()
driver.get(links[8])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})


def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,5): 
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 4:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)

for bus in bus_data:
    print(bus)
driver.quit()

bus_dataframe9 = pd.DataFrame(bus_data)
bus_dataframe9.to_csv('bus_dataframe9.csv', index=False)

############################################################NO.9 KAAC##################################################

driver = webdriver.Chrome()
driver.get(links[9])
time.sleep(5)

urls_list = get_route_links(driver)
df = pd.DataFrame({'URLs': urls_list})

def navigate_and_collect_links(driver):
    all_links = []
    for page_number in range(1,3): 
        try:
            links_on_page = get_route_links(driver)
            all_links.extend(links_on_page)
            if page_number < 2:  # Only navigate if there are more pages
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page_number + 1}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
                time.sleep(1)
                print(f"Clicking on Page {page_number + 1}")
                next_page_button.click()
                WebDriverWait(driver, 10).until(EC.text_to_be_present_in_element(
                    (By.XPATH, '//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'),
                    str(page_number + 1)))
                print(f"Successfully navigated to page {page_number + 1}")
                time.sleep(3)
        except Exception as e:
            print(f"An error occurred while navigating to page {page_number + 1}: {e}")
            break
    return all_links

all_links = navigate_and_collect_links(driver)
bus_data = extract_bus_data(driver, all_links)


for bus in bus_data:
    print(bus)
    
driver.quit()

bus_dataframe10 = pd.DataFrame(bus_data)
bus_dataframe10.to_csv('bus_dataframe10.csv', index=False)




Clicking on Page 2
Successfully navigated to page 2
Clicking on Page 3
Successfully navigated to page 3
Clicking on Page 4
Successfully navigated to page 4
{'route_link': 'https://www.redbus.in/bus-tickets/solapur-to-goa', 'bus_name': 'Kadamba Transport Corporation Limited (KTCL) - 61005', 'bus_type': 'Non AC Seater 2+2', 'departure_time': '20:00', 'reaching_time': '10:00', 'duration': '14h 00m', 'seats_available': '38 Seats available', 'price': 'INR 850', 'star_rating': '3.1'}
{'route_link': 'https://www.redbus.in/bus-tickets/solapur-to-goa', 'bus_name': 'S R Travels', 'bus_type': 'A/C Seater/Sleeper (2+1)', 'departure_time': '18:45', 'reaching_time': '07:00', 'duration': '12h 15m', 'seats_available': '40 Seats available', 'price': 'INR 700', 'star_rating': '2.6'}
{'route_link': 'https://www.redbus.in/bus-tickets/solapur-to-goa', 'bus_name': 'Humsafar Travels', 'bus_type': 'A/C Sleeper (2+1)', 'departure_time': '23:30', 'reaching_time': '09:00', 'duration': '09h 30m', 'seats_available

In [4]:
import pandas as pd
bus_data = pd.read_csv('FINAL1.csv')


In [10]:
import re

def extract_route_name(url):
    pattern = r'/bus-tickets/([a-zA-Z-]+)'
    match = re.search(pattern, url)
    if match:
        route_name = match.group(1)
        return route_name.title()
    return None


# Apply the function to the route_link column
bus_data['route_name'] = bus_data['route_link'].apply(extract_route_name)

# Print the resulting DataFrame
print(bus_data)


                                             route_link  \
0      https://www.redbus.in/bus-tickets/solapur-to-goa   
1      https://www.redbus.in/bus-tickets/solapur-to-goa   
2      https://www.redbus.in/bus-tickets/solapur-to-goa   
3      https://www.redbus.in/bus-tickets/solapur-to-goa   
4     https://www.redbus.in/bus-tickets/goa-to-kolha...   
...                                                 ...   
2933  https://www.redbus.in/bus-tickets/manja-assam-...   
2934  https://www.redbus.in/bus-tickets/manja-assam-...   
2935  https://www.redbus.in/bus-tickets/manja-assam-...   
2936  https://www.redbus.in/bus-tickets/hamren-to-diphu   
2937  https://www.redbus.in/bus-tickets/hamren-to-diphu   

                                               bus_name  \
0     Kadamba Transport Corporation Limited (KTCL) -...   
1                                           S R Travels   
2                                      Humsafar Travels   
3                                    SR Tours & Travels

In [11]:
import pandas as pd
df=pd.DataFrame(bus_data)

In [12]:
df.to_csv('FINALFINAL.csv', index=False)

In [14]:
df

,route_link,bus_name,bus_type,departure_time,reaching_time,duration,seats_available,price,star_rating,route_name
0,https://www.redbus.in/bus-tickets/solapur-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,20:00,10:00,14h 00m,38 Seats available,INR 850,3.1,Solapur-To-Goa
1,https://www.redbus.in/bus-tickets/solapur-to-goa,S R Travels,A/C Seater/Sleeper (2+1),18:45,07:00,12h 15m,40 Seats available,INR 700,2.6,Solapur-To-Goa
2,https://www.redbus.in/bus-tickets/solapur-to-goa,Humsafar Travels,A/C Sleeper (2+1),23:30,09:00,09h 30m,27 Seats available,INR 1250,3.0,Solapur-To-Goa
3,https://www.redbus.in/bus-tickets/solapur-to-goa,SR Tours & Travels,A/C Sleeper (2+1),18:00,05:30,11h 30m,30 Seats available,1080,NaN,Solapur-To-Goa
4,https://www.redbus.in/bus-tickets/goa-to-kolha...,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,19:00,02:30,07h 30m,41 Seats available,INR 420,4.0,Goa-To-Kolhapur-Maharashtra
...,...,...,...,...,...,...,...,...,...,...
2933,https://www.redbus.in/bus-tickets/manja-assam-...,Kareng Travels,Non AC Seater (2+2),04:45,09:30,04h 45m,45 Seats available,342,2.9,Manja-Assam-To-Guwahati
2934,https://www.redbus.in/bus-tickets/manja-assam-...,Kareng Travels,A/C Seater (2+1),05:00,10:00,05h 00m,31 Seats available,INR 400,3.9,Manja-Assam-To-Guwahati
2935,https://www.redbus.in/bus-tickets/manja-assam-...,Kareng Travels,Non AC Seater (2+1),11:30,16:30,05h 00m,37 Seats available,INR 360,3.8,Manja-Assam-To-Guwahati
2936,https://www.redbus.in/bus-tickets/hamren-to-diphu,KAAC TRANSPORT - 175491,Non AC Seater (2+2),05:30,10:20,04h 50m,41 Seats available,INR 246,2.6,Hamren-To-Diphu


In [15]:
df.isnull().sum()


route_link          0
bus_name            1
bus_type            3
departure_time      1
reaching_time       1
duration            1
seats_available     1
price               1
star_rating        91
route_name          0
dtype: int64

In [16]:
df.fillna(0, inplace=True)



In [17]:
df.isnull().sum()

route_link         0
bus_name           0
bus_type           0
departure_time     0
reaching_time      0
duration           0
seats_available    0
price              0
star_rating        0
route_name         0
dtype: int64

In [ ]:
#####################################################################SQL######################################################

In [18]:
import pymysql

In [19]:
myconnection = pymysql.connect(host='127.0.0.1',user='root',password='root')

In [20]:
myconnection.cursor().execute("create database redbus")

1

In [21]:
a=",".join(f"{i} {j}"
for i,j in zip(df.columns,df.dtypes)).replace("float64","float").replace("category","text").replace("int64","int").replace("object","text")

In [22]:
f"create table bus ({a})"

'create table bus (route_link text,bus_name text,bus_type text,departure_time text,reaching_time text,duration text,seats_available text,price text,star_rating float,route_name text)'

In [23]:
myconnection.cursor().execute(f"create table redbus.bus ({a})")

0

In [24]:
len(df)

2938

In [25]:
sql = "insert into redbus.bus values"
for i in range(len(df)):
    myconnection.cursor().execute(f"{sql} {tuple(df.iloc[i])}")
    myconnection.commit()

In [27]:
df

,route_link,bus_name,bus_type,departure_time,reaching_time,duration,seats_available,price,star_rating,route_name
0,https://www.redbus.in/bus-tickets/solapur-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,20:00,10:00,14h 00m,38 Seats available,INR 850,3.1,Solapur-To-Goa
1,https://www.redbus.in/bus-tickets/solapur-to-goa,S R Travels,A/C Seater/Sleeper (2+1),18:45,07:00,12h 15m,40 Seats available,INR 700,2.6,Solapur-To-Goa
2,https://www.redbus.in/bus-tickets/solapur-to-goa,Humsafar Travels,A/C Sleeper (2+1),23:30,09:00,09h 30m,27 Seats available,INR 1250,3.0,Solapur-To-Goa
3,https://www.redbus.in/bus-tickets/solapur-to-goa,SR Tours & Travels,A/C Sleeper (2+1),18:00,05:30,11h 30m,30 Seats available,1080,0.0,Solapur-To-Goa
4,https://www.redbus.in/bus-tickets/goa-to-kolha...,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,19:00,02:30,07h 30m,41 Seats available,INR 420,4.0,Goa-To-Kolhapur-Maharashtra
...,...,...,...,...,...,...,...,...,...,...
2933,https://www.redbus.in/bus-tickets/manja-assam-...,Kareng Travels,Non AC Seater (2+2),04:45,09:30,04h 45m,45 Seats available,342,2.9,Manja-Assam-To-Guwahati
2934,https://www.redbus.in/bus-tickets/manja-assam-...,Kareng Travels,A/C Seater (2+1),05:00,10:00,05h 00m,31 Seats available,INR 400,3.9,Manja-Assam-To-Guwahati
2935,https://www.redbus.in/bus-tickets/manja-assam-...,Kareng Travels,Non AC Seater (2+1),11:30,16:30,05h 00m,37 Seats available,INR 360,3.8,Manja-Assam-To-Guwahati
2936,https://www.redbus.in/bus-tickets/hamren-to-diphu,KAAC TRANSPORT - 175491,Non AC Seater (2+2),05:30,10:20,04h 50m,41 Seats available,INR 246,2.6,Hamren-To-Diphu


In [ ]:
df.isnull().sum()